# Unmatched multiomic data from PBMC
We will work through the example of 10X multi-omics data from Peripheral Blood Mononuclear Cells.  The data are freely available from 10X and we supply another jupyter notebook downloads the data from 10X and counts the unspliced and spliced transcripts.

### This notebook broaches the problem of unmatched multiomic data and is preliminary. Integrating unmatched data is computationally more time consuming.

### Preliminaries
We begin in the usual fashion by importing needed packages.  We load modules specific to MultiDynamo here.  Classes and methods of MultiDynamo have all been added to a single folder 'multi' that is parallel to 'preprocessing', 'estimation', 'prediction', etc.  When fully incorportated into dynamo this will disappear.

In [1]:
import warnings
warnings.simplefilter('ignore', category=Warning)

import dynamo as dyn
import matplotlib.pyplot as plt
import muon as mu
import numpy as np
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
import scanpy as sc
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, frameon=False, facecolor='white')
from scipy.sparse import issparse, spmatrix
import sys

# MultiVelo related classes and methods - This will disappear if we fully incorporate into dynamo
PARENT_DIR = Path().resolve().parent
sys.path.append(os.path.join(PARENT_DIR, 'dynamo', 'dynamo', 'multi'))

from ATACseqTools import annotate_integrated_mdata, gene_activity, integrate
from MultiIO import read_10x_multiome_h5, read_10x_atac_rna_h5
from MultiPreprocessor import MultiPreprocessor
from MultiomicVectorField import MultiomicVectorField
from MultiVelocity import MultiVelocity

scanpy==1.10.2 anndata==0.10.8 umap==0.5.6 numpy==1.26.4 scipy==1.14.0 pandas==2.2.2 scikit-learn==1.4.2 statsmodels==0.14.2 igraph==0.10.8 louvain==0.8.0 pynndescent==0.5.13


### Define paths
Here we define paths in a locale specific manner. I run on both OSX and Linux platforms with synching of data and results through the cloud; globals.py defines global variables in a platform aware manner.

You can either modify globals.py or just specify:
- Location of the MultiDynamo installation (BASE_PATH)
- Location of cache for intermediate results (CACHE_PATH)
- Location(s) of cellranger style 'outs' directory(ies) containing the counts data (MULTIOME_PATH or ATAC_PATH and RNA_PATH)
- Location to reference data, i.e. annotation in GTF format (GTF_PATH)

In [2]:
# Read in global variables from file
from globals import ATAC_PATH, CACHE_PATH, GTF_PATH, MULTIOME_PATH, RNA_PATH 

# ALTERNATIVELY, YOU CAN DEFINE THE FOLLOWING GLOBAL VARIABLES BELOW (DEFINE AND UNCOMMENT AS NEEDED)
# BASE_PATH = /your/MultiDynamo/path (should contain dynamo and multi)

# ... path to cache for intermediate results
# CACHE_PATH = /your/path/to/cache

# ... for matched multiomic data
# MULTIOME_PATH = /your/path/to/multiome/outs

# ... for unmatched multiomic data
# ATAC_PATH = /your path to/atac/outs
# RNA_PATH = /your/path/to/rna/outs

# ... path to genome annotation
# GTF_PATH = /your/local/human/genome/gtf

## Load data
Multi-omic data comes in one of two forms generally: (1) Matched data from multiple modaliies for each cell or (2) Unmatched data from different modalities for different cells.  The latter requires additional integration steps.  We'll begin in full generality with both cases, but only fully demonstrate the matched case.  We'll provide another jupyter notebook for the (more time consuming) unmatched case.

The input data will be (one or two) 'outs' folders containing cellranger output and in parallel a velocyto directory containing the loom file of spliced and uncpliced reads as computed by velocyto.

### Unmatched multimodal data
Here we demonstrate the data loader for the case where scATAC-seq and scRNA-seq data were acquired from cells separately.

***Note: This automatically reads in a loom file of splicing counts computed by velocyto.***

In [3]:
# Read in unmatched data
unmatched_mdata = read_10x_atac_rna_h5(atac_path=ATAC_PATH, rna_path=RNA_PATH)

|-----> Deserializing UNMATCHED scATAC-seq and scRNA-seq data ...
|-----------> reading scATAC-seq data
|-----------> reading scRNA-seq data
|-----------> combining scATAC-seq data and scRNA-seq data into MuData object ...
|-----------------> <insert> .uns[matched_atac_rna_data] = False
|-----------------> <insert> path to fragments file in .uns['files']
|-----------> adding peak annotation ...
|-----------> homogenizing cell indices ...
|-----------> adding splicing data ...
|-----> [read_10x_atac_rna_h5] completed [92.1638s]


#### Checkpoint
We serialize unmatched data to cache in the form of an .h5mu file.

In [4]:
unmatched_mdata.write(os.path.join(CACHE_PATH, 'unmatched_mdata.h5mu'))

## Pre-processing
Here we carry out preprocessing as specified in recipes in the manner carried out by dynamo.  Preprocessing of scATAC-seq data involves computing latent semantic indexing, followed by filtering of cells and peaks.  Preprocessing of scRNA-seq data uses functionality already build into dynamo.

In [5]:
# Instantiate a multi-omic preprocessor
multi_preprocessor = MultiPreprocessor(cell_cycle_score_enable=False)

# Preprocess
multi_preprocessor.preprocess_mdata(unmatched_mdata,
                                    recipe_dict={'atac': 'muon', 'rna': 'monocle'})

|-----> Running muon preprocessing pipeline for scATAC-seq data ...


[W::hts_idx_load3] The index file is older than the data file: /data/LIRGE/external_data/10k_human_PBMC_ATAC/outs/fragments.tsv.gz.tbi
Reading Fragments: 100%|██████████████████████████████████████████████████████████| 1000000/1000000 [00:04<00:00, 233486.91it/s]


|-----------> filtered out 0 outlier features
|-----------> filtered out 206 outlier cells
|-----------> filtered out 202 outlier cells
|-----> computing TF-IDF
|-----> normalizing
|-----> feature selection
|-----------> identified 10116 highly variable features
|-----> computing latent sematic indexing
|-----------> <insert> X_lsi key in .obsm
|-----------> <insert> LSI key in .varm
|-----------> <insert> [lsi][stdev] key in .uns
|-----> [preprocess_atac_muon] completed [269.5963s]
|-----> Running monocle preprocessing pipeline...
|-----------> filtered out 36 outlier cells
|-----------> filtered out 15349 outlier genes
|-----> PCA dimension reduction
|-----> <insert> X_pca to obsm in AnnData Object.
|-----> [Preprocessor-monocle] completed [93.6963s]


#### Checkpoint
We serialize preprocessed object to cache.

In [6]:
unmatched_mdata.write(os.path.join(CACHE_PATH, 'preprocessed_unmatched_mdata.h5mu'))

## Integration
Integrating unmatched multi-omic data is more difficult and we have implemented several versions.  Here we demonstrate an approach via MultiVI which is comutationally intensive.

#### Recovery from checkpoint
The next block deserializes the preprocessed object from cache.  It may be skipped if the object is already memory resident.

In [7]:
unmatched_mdata = mu.read(os.path.join(CACHE_PATH, 'unmatched_mdata.h5mu'))

In [ ]:
integrated_unmatched_mdata = integrate(mdata = unmatched_mdata,
                                       gtf_path = GTF_PATH)

|-----> Integration via MULTIVI ...
|-----------> Computing gene activities
|-----------------> reading GTF
|-----------------> extending genes to estimate regulatory regions
|-----------------> [extend_genes] completed [44.2556s]
|-----------------> overlapping peaks and extended genes
|-----------------> building dictionaries


Processing peaks: 100%|████████████████████████████████████████████████████████████████| 164487/164487 [26:09<00:00, 104.80it/s]

|-----------------> aggregating results


|-----------------> [aggregating_results] completed [81.4242s]
|-----------> Preparing ATAC-seq data for MULTIVI
|-----------> Preparing RNA-seq data for MULTIVI
|-----------> Setting up combined data for MULTIVI


Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: CUDA
Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


|-----------> Instantiating MULTIVI model
|-----------> Training MULTIVI model


INFO: GPU available: True (cuda), used: True
GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 13/500:   2%|▎             | 12/500 [06:51<4:37:06, 34.07s/it, v_num=1, train_loss_step=2.41e+4, train_loss_epoch=2.47e+4]

#### Checkpoint
We serialize the integrated and preprocessed object to cache.

In [ ]:
integrated_unmatched_mdata.write(os.path.join(CACHE_PATH, 'integrated_unmatched_mdata.h5mu'))

## Annotate cells
We use CellTypist for automated cell type annotation of the cells.

We'll filter out everything except for T cells, since we are primarily interested in T cell development.

In [ ]:
integrated_unmatched_mdata = annotate_integrated_mdata(integrated_unmatched_mdata)

In [ ]:
# Extract the predicted cell types
cell_types = list(set(integrated_unmatched_mdata.mod['rna'].obs['cell_type']))
t_cell_types = [cell_type for cell_type in cell_types if 'T cell' in cell_type]
mask = [cell_type in t_cell_types for cell_type in integrated_unmatched_mdata.mod['rna'].obs['cell_type']]

# Filter the cells to T cells
integrated_unmatched_mdata = integrated_unmatched_mdata[mask, :].copy()

In [ ]:
integrated_unmatched_mdata.write(os.path.join(CACHE_PATH, 'annotated_integrated_unmatched_mdata.h5mu'))

## Compute multi-omic velocity
For matched data we use the linkage files computed by cellranger to narrow down list of cis-regulatory elements considered for dynamics.  We are still testing several implementations of our own.

In [17]:
# Read in annotated and integrated MuData object
integrated_unmatched_mdata = mu.read(os.path.join(CACHE_PATH, 'annotated_integrated_unmatched_tcell_mdata.h5mu'))

In [ ]:
# Compute MultiVelocity from the annotated and integrated MuData object
#  ***************** DO NOT RUN ***************************************
multi_velocity = MultiVelocity(mdata=integrated_unmatched_mdata,
                               linkage_method='cellranger', # future: 'cicero' or 'scenic+'
                               neighbor_method='wnn') # 'wnn' or 'multivi')

multi_velocity.compute_velocities(num_processes=6)

We can convert MultiVelocity to a MuData object, serialize and read it back from disk.

In [19]:
# Serialize multi_velocity to disk - this involved conversion to MuData object
#  ***************** DO NOT RUN ***************************************
multi_velocity.write(os.path.join(CACHE_PATH, 'multi_velocity_tcell_adata.h5mu'))

## Reconstruction of the vector field
Here we start after a checkpoint, deserializing previously serialized computations.  You can skip this block if the MultiVelocity object is already memory resident.

In [5]:
# Read the MuData object
#  ***************** DO NOT RUN ***************************************
multi_velocity_md = mu.read(os.path.join(CACHE_PATH, 'multi_velocity_tcell_adata.h5mu'))

# Instantiate the MultiVelocity object
multi_velocity = MultiVelocity.from_mdata(multi_velocity_md)

Instantiate a MultiomicVectorField object from the MultiVelocity object.

In [6]:
#  ***************** DO NOT RUN ***************************************
multi_vector_field = MultiomicVectorField(multi_velocity)

#### Checkpoint:
We can extract from the MultiVectorField class an AnnData object that is suitable for further analysis using dynamo alone.  We'll extract it and save it in the block below.

In [7]:
#  ***************** DO NOT RUN ***************************************
multi_vector_field.multi_adata.write(os.path.join(CACHE_PATH, 'multi_adata.h5ad'))

### Computation of the smooth analytic vector field

Or we can proceed on with methods from Multi

In [ ]:
#  ***************** DO NOT RUN ***************************************
multi_adata = multi_vector_field.cell_velocities()

In [ ]:
#  ***************** DO NOT RUN ***************************************
multi_vector_field.compute_vector_field()

### Visualization of Velocities

In [ ]:
#  ***************** DO NOT RUN ***************************************
import matplotlib.pyplot as plt
%matplotlib inline

multi_vector_field.plot_cell_wise_vectors()
multi_vector_field.plot_streamline_plot()

## Characterization of vector field topology

In [ ]:
#  ***************** DO NOT RUN ***************************************
multi_vector_field.plot_topography()

## In silico perturbation
First we show the effects of a perturbation to a gene.

We begin by listing the genes that we can perturb:

In [ ]:
#  ***************** DO NOT RUN ***************************************
# Extract information about all genes and CREs
gene_and_cre_df = multi_vector_field.multi_adata.var.copy()

# Narrow to perturbable genes
gene_df = gene_and_cre_df[gene_and_cre_df['feature_type']  == 'gene']
perturbable_gene_df = gene_df[gene_df['use_for_pca']]

# We can search for genes starting with certain letters - Interleukins are generally interesting ...
perturbable_gene_df[perturbable_gene_df.index.str.startswith('IL')]

In [ ]:
#  ***************** DO NOT RUN ***************************************
# Compute the perturbation to IL2 receptor alpha expresion - important in autoimmunity
multi_vector_field.predict_perturbation(gene='IL2RA',
                                        expression=50)

In [ ]:
#  ***************** DO NOT RUN ***************************************
# Extract information about all genes and CREs
gene_and_cre_df = multi_vector_field.multi_adata.var.copy()

# Narrow to perturbable CREs
cre_df = gene_and_cre_df[gene_and_cre_df['feature_type']  == 'CRE']
perturbable_cre_df = cre_df[cre_df['use_for_pca']]

# We can search for CRE on specific chromosome - IL2RA is on chr10 ...
perturbable_cre_df[perturbable_cre_df.index.str.startswith('chr10')]

### Perturbations to cis regulatory elements
Here we demonstrate a computation of a perturbation to a cis-regulatory element.

In [ ]:
#  ***************** DO NOT RUN ***************************************
multi_vector_field.predict_perturbation(gene='chr10:114542903-114545289',
                                        expression=50)